In [1]:
import cv2
import mediapipe as mp
import math

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [3]:
def distancia(p1, p2):
    return math.sqrt((p2.x - p1-x)**2 + (p2.y - p1.y)**2)

In [17]:
# Función para determinar si el pulgar está levantado, tip: punta del dedo, dip: parte media, mcp: parte inferior
def dedo_levantado(landmarks, dedo_tip, dedo_dip, dedo_pip, dedo_mcp, umbral=0.1):
    #Compara la posición "y" de la punta con respecto a las anteriores
    return(landmarks[dedo_tip].y < landmarks[dedo_dip].y and 
           landmarks[dedo_dip].y < landmarks[dedo_pip].y and 
           landmarks[dedo_pip].y < landmarks[dedo_mcp].y)

In [27]:
ruta = "mano2.jpg" 

In [28]:
with mp_hands.Hands(
    static_image_mode = True,
    max_num_hands=2,
    min_detection_confidence = 0.5
) as hands:
    image = cv2.imread(ruta)
    height, width,_ = image.shape
    image = cv2.flip(image, 1)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    resultados = hands.process(image_rgb)

    print("handedness: ", resultados.multi_handedness)

handedness:  [classification {
  index: 1
  score: 0.9522156715393066
  label: "Right"
}
]


In [31]:
if resultados.multi_hand_landmarks:
    for hands_landmarks in resultados.multi_hand_landmarks:
        #Dibujar las marcas y conexiones
        mp_drawing.draw_landmarks(image, hands_landmarks, mp_hands.HAND_CONNECTIONS)

        #Obtener las marcas como una lista
        landmarks = hands_landmarks.landmark

        # Media pipe define los indices de la punta de los dedos según mediapipe
        PULGAR_TIP = 4
        INDICE_TIP = 8
        MEDIO_TIP = 12
        ANULAR_TIP = 16
        MENIQUE_TIP = 20

        # Puntos para cada dedo (tip, dip, pip, mcp)
        dedos = {
            "Pulgar": (PULGAR_TIP, 2, 1, 0),
            "Indice": (INDICE_TIP, 6, 5, 0),
            "Medio": (MEDIO_TIP, 10, 9, 0),
            "Anular": (ANULAR_TIP, 14, 13, 0),
            "Menique": (MENIQUE_TIP, 18, 17, 0)
        }

        # Verificar que dedos estan levantados
        dedos_levantados = []
        for nombre, (tip, dip, pip, mcp) in dedos.items():
            if dedo_levantado(landmarks, tip, dip, pip, mcp):
                dedos_levantados.append(nombre)

        print("Dedos levantados: ", dedos_levantados)

        #Mostrar los dedos levantados en la imgaen
        texto = f"Dedos:{','.join(dedos_levantados)}"
        cv2.putText(image, texto, (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,255,0),2)

        #Mostrar imagen resultante
        cv2.imshow("Imagen", image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

Dedos levantados:  ['Pulgar', 'Indice', 'Medio', 'Anular', 'Menique']
